# Детекция и локализация объектов на изображении.



На прошлых занятиях мы говорили про классификацию (распознавание) изображений. Мы подавали на вход нейронной сети некоторое изображение, а она определяла, что на нем нарисовано.

Но ведь может быть и так, что  на одном изображении нарисовано несколько объектов вместе, например, котик и собачка. Что же в таком случае определит нейронная сеть? Проверьте самостоятельно.



Если объектов на изображении несколько, то те архитектуры, которыми мы пользовались ранее для классификации, нам не помогут, нужны другие подходы.

Самое простое, что можно здесь придумать - а давайте разобьем изображение на отдельные окошки и будем распознавать только то, что содержится в окошке, а не все изображение целиком. Тогда для каждого окошка нейронная сеть сообщит нам, какой объект в нем нарисован. Потом мы объединим показания из окошек, если, например, в двух близкорасположенных окошках нарисована кошка, то, наверное, это одна и та же кошка.

Объекты на изображении могут быть разного размера. Если оно получено путем фотографирования, то близкорасположенные объекты на фотографии больше, чем далекие. Значит нам надо перебирать не только окошки с разным положением, но и с разным масштабом.

Окошки, на которые мы разбиваем изображение, могут быть и пересекающиеся, наверное, для тех окошек, в которые попадает объект целиком, нейронная сеть даст большой уровень уверенности распознавания. А для тех, в которых попадает только часть объекта, этот уровень будет маленьким. Тогда сравнивая уровни уверенности для разных окошек, можем выбрать такое, для которого уровень больше и считать его окошком, содержащим распознанный объект. Получается, мы не только распознали каждый объект на изображении, но и нашли окошки, их содержащие.

Процесс, когда мы рассчитываем положение окошка, содержащего объект, называют **локализация** (localization), а процесс, когда мы находим такие объекты (с окошками) на изображении - **детекция** (детектирование, detection) объектов. Это задачи похожие на распознавание, но отличаются тем, что кроме указания класса, надо сообщить и окошко, в котором содержится объект. Окошки обычно прямоугольные, мы можем задать их координатами, скажем, левого верхнего и нижнего правого углов, или координатами центра и шириной и высотой.

Более сложный вариант задачи, когда кроме окошка надо указать, какие именно пиксели окошка относятся к объекту, а какие нет - называют (объектовой) **сегментацией** (instance segmentation).

![img](https://drive.google.com/uc?id=1_9HaQzJQgv7iAFWVK1Z71JgrZHjMmiHG)



Есть две идеи работы с такими окошками-кандидатами на проверку:

- использовать некоторый алгоритм или обучать нейронную сеть выбору положения и размера окошек (сеть SSD и семейство сетей R-CNN);
- выбрать положение и размер окошек заранее (сеть YOLO) и проверять есть ли там объект, уточняя положение окошка при необходимости.

Первый случай дает потенциально точность, второй - быстроту.

В обоих случаях логично работать не с самими изображениями, а с их картами признаков, полученными после прохождения изображения через какую-нибудь нейронную сеть классификации, такую сеть называют **основой** (backbone).




## Оценка похожести окошек. IoU.

Как сравнивать классы мы уже знаем и делали это. Но как сравнить окошки, какое из них лучше, какое хуже?

Пусть есть два окошка, настоящее и предсказанное нами. Когда они полностью совпадают это очень хорошо. Когда не пересекаются - очень плохо. Когда пересекаются частично это лучше, чем если вообще не совпадают. Но и размер окошек тоже важен, если одно маленькое и находится внутри другого, то оно верно показывает положение объекта, но неверно его размер. И такие случаи нужно предусмотреть.

Для сравнения окошек используют величину IoU (Intersect over Union, пересечение над объединением), которая считается так:
- находят площадь пересечения двух окошек
- и делят это на площадь объединения двух окошек.

![img](https://drive.google.com/uc?id=1SBd9_iZj38eawzTEpG15rDKTfgVrhNC1)

Проверьте, что действительно выполняются желаемые нами свойства:
- если окошки совпадают, пересечение и объединение одинаковы и IoU=1,
- если окошки не пересекаются, то площадь пересечения равна 0 и IoU=0.
- если одно окошко внутри другого, или они частично пересекаются то 0 < IoU < 1

Функция ошибки для минимизации это 1-IoU.

Но тут есть проблема, если окошки не пересекаются вообще, то мы никак неузнаем, как менять параметры сети, чтобы их сблизить, ведь производная будет 0. Надо добавить дополнительные члены в функцию ошибки, которые показывают дальность окошек друг от друга. Также надо учесть разницу в отношении сторон окошек. Все это можно сделать, но, конечно, это усложнит формулы для вычисления функции ошибки (мы не будем приводить из здесь).








# Заключение
Подходы к детекции объектов на изображении разнообразны. Упомянем сети семейства R-CNN (Region-based CNN), но познакомимся с ними в теме про сегментацию.

Посмотреть более современные решения, результаты соревнований по детекции можно [здесь](https://paperswithcode.com/task/object-detection).

# Ссылки

Использованы материалы:
- https://jonathan-hui.medium.com/yolov4-c9901eaa8e61
- https://towardsdatascience.com/understanding-ssd-multibox-real-time-object-detection-in-deep-learning-495ef744fab
- https://github.com/pjreddie/darknet/issues/1439
- https://github.com/AlexeyAB/darknet/

- https://colab.research.google.com/github/d2l-ai/d2l-en-colab/blob/master/chapter_computer-vision/rcnn.ipynb
- https://colab.research.google.com/github/tensorflow/models/blob/master/research/object_detection/colab_tutorials/context_rcnn_tutorial.ipynb
- https://colab.research.google.com/github/google/eng-edu/blob/master/ml/pc/exercises/image_classification_part3.ipynb#scrollTo=YHK6DyunSbs4

- https://d2l.ai/chapter_computer-vision/ssd.html
- https://d2l.ai/chapter_computer-vision/rcnn.html
